In [32]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import vggish_slim, vggish_params, vggish_input
import tensorflow as tf
from sklearn.model_selection import train_test_split
# Disable eager execution
tf.compat.v1.disable_eager_execution()
import librosa
import soundfile as sf
from IPython.display import clear_output

In [2]:
class DataGenerator(Sequence):
    def __init__(self, directory, batch_size=32, shuffle=True):
        self.directory = directory
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.file_paths = []
        self.labels = []
        self.class_names = ['porn', 'normal']
        for class_index, class_name in enumerate(self.class_names):
            class_dir = os.path.join(directory, class_name)
            if os.path.isdir(class_dir):
                for file_name in os.listdir(class_dir):
                    self.file_paths.append(os.path.join(class_dir, file_name))
                    self.labels.append(class_index)
        self.on_epoch_end()
        self.sess, self.features_tensor, self.embeddings_tensor = self.load_vggish_model()

    def load_vggish_model(self):
        embeddings_tensor = vggish_slim.define_vggish_slim(training=False)
        sess = tf.compat.v1.Session()
        vggish_slim.load_vggish_slim_checkpoint(sess, 'vggish_model.ckpt')
        features_tensor = sess.graph.get_tensor_by_name(vggish_params.INPUT_TENSOR_NAME)
        return sess, features_tensor, embeddings_tensor

    def __len__(self):
        return int(np.floor(len(self.file_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_paths = self.file_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__data_generation(batch_paths, batch_labels)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.file_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_paths, batch_labels):
        X = np.empty((self.batch_size, 128))  # VGGish embeddings are 128-D
        y = np.empty((self.batch_size), dtype=int)
        for i, (file_path, label) in enumerate(zip(batch_paths, batch_labels)):
            examples_batch = vggish_input.wavfile_to_examples(file_path)
            [embeddings] = self.sess.run([self.embeddings_tensor], feed_dict={self.features_tensor: examples_batch})
            embeddings_mean = np.mean(embeddings, axis=0)  # Average embeddings across examples
            X[i,] = embeddings_mean
            y[i] = label
        y = tf.keras.utils.to_categorical(y, num_classes=len(self.class_names))
        return X, y

In [34]:
from __future__ import print_function

from random import shuffle

import numpy as np
import tensorflow.compat.v1 as tf
import tf_slim as slim

import vggish_input
import vggish_params
import vggish_slim

In [28]:
def _get_file_paths_and_labels(porn_dir, normal_dir):
    """Collects file paths and assigns labels based on the directory structure."""
    file_paths = []
    labels = []
    
    # Process porn directory
    for file_name in os.listdir(porn_dir):
        if file_name.endswith('.wav'):
            file_paths.append(os.path.join(porn_dir, file_name))
            labels.append(1)  # Label for porn class
    
    # Process normal directory
    for file_name in os.listdir(normal_dir):
        if file_name.endswith('.wav'):
            file_paths.append(os.path.join(normal_dir, file_name))
            labels.append(0)  # Label for normal class
    
    return file_paths, labels


def _get_real_audio_examples_batch(file_paths, labels, batch_size):
    """Returns a shuffled batch of examples from real audio files.

    Args:
        file_paths (list of str): List of file paths to the audio files.
        labels (list of int): List of corresponding labels.
        batch_size (int): Number of samples in the batch.
        sr (int): Sampling rate for audio files.
        n_mels (int): Number of mel bands to generate.
        n_frames (int): Number of frames in each mel spectrogram.

    Returns:
        tuple: (features, labels) where features is a NumPy array of shape
        [batch_size, num_frames, num_mels] and labels is a NumPy array of shape
        [batch_size, num_classes].
    """
    # Ensure the batch size does not exceed the number of available samples
    batch_size = min(batch_size, len(file_paths))
    
    # Randomly select batch_size number of file paths and their corresponding labels
    selected_indices = np.random.choice(len(file_paths), batch_size, replace=False)
    batch_paths = [file_paths[i] for i in selected_indices]
    batch_labels = [labels[i] for i in selected_indices]

    features = []
    one_hot_labels = []

    for file_path, label in zip(batch_paths, batch_labels):
        # Load the audio file
        y= vggish_input.wavfile_to_examples(file_path)
        
        features.append(y)

        # Convert label to one-hot
        one_hot_label = tf.keras.utils.to_categorical(label, num_classes=2)
        one_hot_labels.append(one_hot_label)

    features = np.array(features)
    one_hot_labels = np.array(one_hot_labels)

    # Shuffle features and labels together
    labeled_examples = list(zip(features, one_hot_labels))
    shuffle(labeled_examples)

    # Separate and return the features and labels
    features = np.array([example for (example, _) in labeled_examples])
    labels = np.array([label for (_, label) in labeled_examples])

    return features, labels

def count_files(directory1, directory2):
    # Initialize counts for each directory
    count = 0

    # Count files in directory 1
    for _, _, files in os.walk(directory1):
        count += len(files)

    # Count files in directory 2
    for _, _, files in os.walk(directory2):
        count += len(files)

    return count

In [31]:
# Example usage
porn_dir = r'A:\AI DB\LSPD\Audio\porn'
normal_dir = r'A:\AI DB\LSPD\Audio\normal'
batch_size = 32
total_files=count_files(porn_dir, normal_dir)
print("total number of samples: ",str(total_files))
steps_per_epochs=int(total_files/batch_size)
print("total number of steps: ",str(steps_per_epochs))
file_paths, labels = _get_file_paths_and_labels(porn_dir, normal_dir)
features, labels = _get_real_audio_examples_batch(file_paths, labels, batch_size)
features=np.squeeze(features, axis=1)
print("Features shape:", features.shape)
print("Labels shape:", labels.shape)

total number of samples:  241340
total number of steps:  7541
Features shape: (32, 96, 64)
Labels shape: (32, 2)


In [40]:
_NUM_CLASSES=2
with tf.Graph().as_default(), tf.Session() as sess:
    # Define VGGish.
    embeddings = vggish_slim.define_vggish_slim(training=False)

    # Define a shallow classification model and associated training ops on top
    # of VGGish.
    with tf.variable_scope('mymodel'):
      # Add a fully connected layer with 100 units. Add an activation function
      # to the embeddings since they are pre-activation.
      num_units = 100
      fc = slim.fully_connected(tf.nn.relu(embeddings), num_units)

      # Add a classifier layer at the end, consisting of parallel logistic
      # classifiers, one per class. This allows for multi-class tasks.
      logits = slim.fully_connected(
          fc, _NUM_CLASSES, activation_fn=None, scope='logits')
      tf.sigmoid(logits, name='prediction')

      # Add training ops.
      with tf.variable_scope('train'):
        global_step = tf.train.create_global_step()

        # Labels are assumed to be fed as a batch multi-hot vectors, with
        # a 1 in the position of each positive class label, and 0 elsewhere.
        labels_input = tf.placeholder(
            tf.float32, shape=(None, _NUM_CLASSES), name='labels')

        # Cross-entropy label loss.
        xent = tf.nn.sigmoid_cross_entropy_with_logits(
            logits=logits, labels=labels_input, name='xent')
        loss = tf.reduce_mean(xent, name='loss_op')
        tf.summary.scalar('loss', loss)

        # We use the same optimizer and hyperparameters as used to train VGGish.
        optimizer = tf.train.AdamOptimizer(
            learning_rate=vggish_params.LEARNING_RATE,
            epsilon=vggish_params.ADAM_EPSILON)
        train_op = optimizer.minimize(loss, global_step=global_step)

    # Initialize all variables in the model, and then load the pre-trained
    # VGGish checkpoint.
    sess.run(tf.global_variables_initializer())
    vggish_slim.load_vggish_slim_checkpoint(sess, 'vggish_model.ckpt')

    # The training loop.
    features_input = sess.graph.get_tensor_by_name(
        vggish_params.INPUT_TENSOR_NAME)
    for _ in range(steps_per_epochs):
        file_paths, labels = _get_file_paths_and_labels(porn_dir, normal_dir)
        (features, labels) = _get_real_audio_examples_batch(file_paths, labels, batch_size)
        features=np.squeeze(features, axis=1)
        [num_steps, loss_value, _] = sess.run(
            [global_step, loss, train_op],
            feed_dict={features_input:features , labels_input: labels})
        clear_output(wait=True)
        percentage = (num_steps / steps_per_epochs) * 100.0
        formatted_percentage = "%.2f%%" % percentage
        print('%s  : loss %g' % (formatted_percentage, loss_value))
    

99.99%  : loss 0.240975


In [10]:
# Disable eager execution
tf.compat.v1.disable_eager_execution()
# Initialize data generator
data_dir = r'A:\AI DB\LSPD\Audio'
batch_size = 32
data_generator = DataGenerator(data_dir, batch_size=batch_size)

# Split the data paths into training and validation sets
file_paths, labels = data_generator.file_paths, data_generator.labels
train_paths, val_paths, train_labels, val_labels = train_test_split(
    file_paths, labels, test_size=0.2, random_state=42)

# Create data generators for training and validation
train_generator = DataGenerator(data_dir, batch_size=batch_size, shuffle=True)
val_generator = DataGenerator(data_dir, batch_size=batch_size, shuffle=False)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(128,)),  # VGGish embeddings are 128-D
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(len(data_generator.class_names), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the generator
model.fit(train_generator, epochs=20, validation_data=val_generator)

# Save the model
model.save('audio_classifier.h5')

INFO:tensorflow:Restoring parameters from vggish_model.ckpt
INFO:tensorflow:Restoring parameters from vggish_model.ckpt
INFO:tensorflow:Restoring parameters from vggish_model.ckpt
Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'vggish_2/input_features' defined at (most recent call last):
    File "C:\ProgramData\anaconda3\envs\amchibrk\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\anaconda3\envs\amchibrk\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Mohamed ali\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\Mohamed ali\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\Mohamed ali\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\Mohamed ali\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loo